<a href="https://colab.research.google.com/github/neelandrakar/Node-URL-shortener/blob/main/Predict_Next_Sales_Test_003.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing Libraries**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

**Importing First dataset**

In [ ]:
sale_dt = pd.read_csv("dealer_wise_sale_with_stock_and_scheme.csv")
sale_dt['purchase_date'] = pd.to_datetime(sale_dt['purchase_date'])
sale_dt['PREV_INVOICE'] = pd.to_datetime(sale_dt['PREV_INVOICE'])

print(sale_dt.head(5))

   dealer_id purchase_date  purchase_rank  pri_sale salescategory_status  \
0       4886    2025-02-18              1     6.090               Normal   
1       4886    2025-02-14              2     9.945               Normal   
2       4886    2025-01-15              3    10.000               Normal   
3       4886    2024-11-27              4     7.000               Normal   
4       8045    2025-02-12              1     6.900               Normal   

   state_id PREV_INVOICE  last_90_days_pri_sale  last_90_days_sec_sale  \
0         5   2025-02-14                 222.42                 224.71   
1         5   2025-01-15                 222.42                 224.31   
2         5   2024-11-27                 222.42                 216.71   
3         5   2024-09-24                 212.22                 200.11   
4         5   2024-12-24                 228.42                 243.81   

   last_90_days_sd_sale  scheme_avalable  
0                  15.0                1  
1           

<ipython-input-38-381f3f81cc07>:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  sale_dt['purchase_date'] = pd.to_datetime(sale_dt['purchase_date'])
<ipython-input-38-381f3f81cc07>:3: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  sale_dt['PREV_INVOICE'] = pd.to_datetime(sale_dt['PREV_INVOICE'])


**Calculating Day Difference**

In [ ]:
sale_dt['DayDiff'] = (sale_dt['purchase_date'] - sale_dt['PREV_INVOICE']).dt.days
print(sale_dt.head(5))

   dealer_id purchase_date  purchase_rank  pri_sale salescategory_status  \
0       4886    2025-02-18              1     6.090               Normal   
1       4886    2025-02-14              2     9.945               Normal   
2       4886    2025-01-15              3    10.000               Normal   
3       4886    2024-11-27              4     7.000               Normal   
4       8045    2025-02-12              1     6.900               Normal   

   state_id PREV_INVOICE  last_90_days_pri_sale  last_90_days_sec_sale  \
0         5   2025-02-14                 222.42                 224.71   
1         5   2025-01-15                 222.42                 224.31   
2         5   2024-11-27                 222.42                 216.71   
3         5   2024-09-24                 212.22                 200.11   
4         5   2024-12-24                 228.42                 243.81   

   last_90_days_sd_sale  scheme_avalable  DayDiff  
0                  15.0                1      

**Calculating mean avg next lift day range**

In [ ]:
sale_day_diff = sale_dt[sale_dt['DayDiff'] >= 10].groupby('dealer_id').agg({'DayDiff': ['mean', 'std']}).reset_index()
sale_day_diff.columns = ['dealer_id', 'DayDiffMean','DayDiffStd']
print(sale_day_diff.head(5))
sale_day_diff = pd.merge(sale_day_diff,sale_dt[['dealer_id', 'scheme_avalable']], on='dealer_id', how='inner')
print(sale_day_diff.head(5))

   dealer_id  DayDiffMean  DayDiffStd
0       4886    47.666667   17.039171
1       8045    53.250000    8.460693
2      11702    16.000000    6.000000
3      12280    53.000000   37.242449
4      12329    17.000000    8.485281
   dealer_id  DayDiffMean  DayDiffStd  scheme_avalable
0       4886    47.666667   17.039171                1
1       4886    47.666667   17.039171                1
2       4886    47.666667   17.039171                1
3       4886    47.666667   17.039171                1
4       8045    53.250000    8.460693                1


**Remove dealers who have less than 3 purchases**

In [ ]:
sale_day_last = sale_dt.drop_duplicates(subset=['dealer_id'], keep='last')

**Dropping NA values**

In [ ]:
sale_day_last = sale_day_last.dropna()
sales_day_last = pd.merge(sale_day_last, sale_day_diff, on='dealer_id')
#print(sale_user)
sale_class = sales_day_last.copy()
sale_class = pd.get_dummies(sale_class)

**Setting NextLiftingRange**

In [ ]:
#print(sale_class.head(5))
sale_class['NextPurchaseDayRange'] = 0
sale_class.loc[sale_class.DayDiffMean > 10, 'NextPurchaseDayRange'] = 1
sale_class.loc[sale_class.DayDiffMean > 20, 'NextPurchaseDayRange'] = 2
sale_class.loc[sale_class.DayDiffMean > 30, 'NextPurchaseDayRange'] = 3
sale_class.loc[sale_class.DayDiffMean > 40, 'NextPurchaseDayRange'] = 4
sale_class.loc[sale_class.DayDiffMean > 50, 'NextPurchaseDayRange'] = 5
sale_class.loc[sale_class.DayDiffMean > 60, 'NextPurchaseDayRange'] = 6
sale_class.loc[sale_class.DayDiffMean > 70, 'NextPurchaseDayRange'] = 7
sale_class.loc[sale_class.DayDiffMean > 80, 'NextPurchaseDayRange'] = 8
sale_class.loc[sale_class.DayDiffMean > 90, 'NextPurchaseDayRange'] = 9
sale_class.loc[sale_class.DayDiffMean > 100, 'NextPurchaseDayRange'] = 10
sale_class.loc[sale_class.DayDiffMean > 110, 'NextPurchaseDayRange'] = 11
sale_class.loc[sale_class.DayDiffMean > 120, 'NextPurchaseDayRange'] = 12
sale_class.loc[sale_class.DayDiffMean > 130, 'NextPurchaseDayRange'] = 13
sale_class.loc[sale_class.DayDiffMean > 140, 'NextPurchaseDayRange'] = 14
sale_class.loc[sale_class.DayDiffMean > 150, 'NextPurchaseDayRange'] = 15
sale_class.loc[sale_class.DayDiffMean > 160, 'NextPurchaseDayRange'] = 16
sale_class.loc[sale_class.DayDiffMean > 170, 'NextPurchaseDayRange'] = 17
sale_class.loc[sale_class.DayDiffMean > 180, 'NextPurchaseDayRange'] = 18
print(sale_class.head(4))

   dealer_id purchase_date  purchase_rank  pri_sale  state_id PREV_INVOICE  \
0       4886    2024-11-27              4       7.0         5   2024-09-24   
1       4886    2024-11-27              4       7.0         5   2024-09-24   
2       4886    2024-11-27              4       7.0         5   2024-09-24   
3       4886    2024-11-27              4       7.0         5   2024-09-24   

   last_90_days_pri_sale  last_90_days_sec_sale  last_90_days_sd_sale  \
0                 212.22                 200.11                  15.0   
1                 212.22                 200.11                  15.0   
2                 212.22                 200.11                  15.0   
3                 212.22                 200.11                  15.0   

   scheme_avalable_x  DayDiff  DayDiffMean  DayDiffStd  scheme_avalable_y  \
0                  1     64.0    47.666667   17.039171                  1   
1                  1     64.0    47.666667   17.039171                  1   
2           

**Importing Mean Avg Stock Dataset**

In [ ]:
mas = pd.read_csv('mean_avg_stock.csv')
print(mas.head(5))

   dealer_id  avg_mean_stock
0    2077497            3.47
1    2165352           33.45
2      40479           73.02
3      39276           36.99
4    2070260           17.35


**Importing Invoice wise other stock dataset**

In [ ]:
invoice_wise_stock_dt =  pd.read_csv('dealer_invoice_wise_stock.csv')
print(invoice_wise_stock_dt.head(5))

   dealer_id purchase_date  purchase_rank  pri_sale PREV_INVOICE  \
0       4886    18-02-2025              1     6.090   14-02-2025   
1       4886    14-02-2025              2     9.945   15-01-2025   
2       4886    15-01-2025              3    10.000   27-11-2024   
3       4886    27-11-2024              4     7.000   24-09-2024   
4       8045    12-02-2025              1     6.900   24-12-2024   

   last_90_days_pri_sale  last_90_days_sec_sale  last_90_days_sd_sale  
0                 222.42                 224.71                  15.0  
1                 222.42                 224.31                  15.0  
2                 222.42                 216.71                  15.0  
3                 212.22                 200.11                  15.0  
4                 228.42                 243.81                   0.0  


**Merging both 2nd and 3rd dataset**

In [ ]:
invoice_wise_stock_bal = pd.merge(invoice_wise_stock_dt, mas, on=['dealer_id'], how='left')
print(invoice_wise_stock_bal.head(5))


   dealer_id purchase_date  purchase_rank  pri_sale PREV_INVOICE  \
0       4886    18-02-2025              1     6.090   14-02-2025   
1       4886    14-02-2025              2     9.945   15-01-2025   
2       4886    15-01-2025              3    10.000   27-11-2024   
3       4886    27-11-2024              4     7.000   24-09-2024   
4       8045    12-02-2025              1     6.900   24-12-2024   

   last_90_days_pri_sale  last_90_days_sec_sale  last_90_days_sd_sale  \
0                 222.42                 224.71                  15.0   
1                 222.42                 224.31                  15.0   
2                 222.42                 216.71                  15.0   
3                 212.22                 200.11                  15.0   
4                 228.42                 243.81                   0.0   

   avg_mean_stock  
0            7.42  
1            7.42  
2            7.42  
3            7.42  
4            8.47  


**Dropping NaN values from invoice_wise_stock_bal**

In [ ]:
invoice_wise_stock_bal = invoice_wise_stock_bal.dropna()


**Checking if stock is less than avg stock**

In [ ]:
# Calculate total_left using columns from invoice_wise_stock_bal
invoice_wise_stock_bal['total_left'] = (
    invoice_wise_stock_bal['last_90_days_pri_sale'] -
    (invoice_wise_stock_bal['last_90_days_sec_sale'] + invoice_wise_stock_bal['last_90_days_sd_sale'])
)

# Add is_stock_low column using vectorized comparison
invoice_wise_stock_bal['is_stock_low'] = (
    invoice_wise_stock_bal['total_left'] < invoice_wise_stock_bal['avg_mean_stock']
).astype(int)

# Drop the specified columns
invoice_wise_stock_bal = invoice_wise_stock_bal.drop(
    columns=['last_90_days_pri_sale', 'last_90_days_sec_sale', 'last_90_days_sd_sale']
)

# Print the updated DataFrame (fixed typo)
print(invoice_wise_stock_bal.head(50))

    dealer_id purchase_date  purchase_rank  pri_sale PREV_INVOICE  \
0        4886    18-02-2025              1     6.090   14-02-2025   
1        4886    14-02-2025              2     9.945   15-01-2025   
2        4886    15-01-2025              3    10.000   27-11-2024   
3        4886    27-11-2024              4     7.000   24-09-2024   
4        8045    12-02-2025              1     6.900   24-12-2024   
5        8045    24-12-2024              2    10.560   27-10-2024   
6        8045    27-10-2024              3     7.035   26-08-2024   
7        8045    26-08-2024              4     9.305   14-07-2024   
8       11702    20-02-2025              1     4.545   15-02-2025   
9       11702    15-02-2025              2    10.855   30-01-2025   
10      11702    30-01-2025              3     3.045   08-01-2025   
11      11702    08-01-2025              4     6.975   29-12-2024   
12      12280    16-02-2025              1     9.020   08-02-2025   
13      12280    08-02-2025       

<ipython-input-49-7a3b917222d3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  invoice_wise_stock_bal['total_left'] = (
<ipython-input-49-7a3b917222d3>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  invoice_wise_stock_bal['is_stock_low'] = (


**Calculating DayDiffMean**

In [ ]:
invoice_wise_stock_bal['purchase_date'] = pd.to_datetime(invoice_wise_stock_bal['purchase_date'])
invoice_wise_stock_bal['PREV_INVOICE'] = pd.to_datetime(invoice_wise_stock_bal['PREV_INVOICE'])


invoice_wise_stock_bal['DayDiff'] = (invoice_wise_stock_bal['purchase_date'] - invoice_wise_stock_bal['PREV_INVOICE']).dt.days

# Merge DayDiffMean into invoice_wise_stock_bal
invoice_wise_stock_bal = invoice_wise_stock_bal.drop(columns=['4'], errors='ignore')  # Drop existing DayDiffMean if any
invoice_wise_stock_bal = invoice_wise_stock_bal.merge(sale_day_diff[['dealer_id', 'DayDiffMean','scheme_avalable']], on='dealer_id', how='inner')







In [ ]:
#invoice_wise_stock_bal.dropna()
print(invoice_wise_stock_bal[invoice_wise_stock_bal['is_stock_low']==0].head(5))


     dealer_id purchase_date  purchase_rank  pri_sale PREV_INVOICE  \
192      14874    2025-02-22              1    13.120   2025-01-31   
193      14874    2025-02-22              1    13.120   2025-01-31   
194      14874    2025-02-22              1    13.120   2025-01-31   
195      14874    2025-02-22              1    13.120   2025-01-31   
196      14874    2025-01-31              2     7.065   2025-01-28   

     avg_mean_stock  total_left  is_stock_low  DayDiff  DayDiffMean  \
192           12.41      15.665             0       22         17.0   
193           12.41      15.665             0       22         17.0   
194           12.41      15.665             0       22         17.0   
195           12.41      15.665             0       22         17.0   
196           12.41      16.665             0        3         17.0   

     scheme_avalable  
192                1  
193                1  
194                1  
195                1  
196                1  


**Setting features and target**

In [ ]:
# Features and target
features = ['dealer_id', 'is_stock_low','scheme_avalable']
x = invoice_wise_stock_bal[features]
y = invoice_wise_stock_bal['DayDiff']
groups = invoice_wise_stock_bal['dealer_id']

**Splitting into group and train**

In [ ]:
from sklearn.model_selection import GroupShuffleSplit
from sklearn.ensemble import RandomForestRegressor

# Grouped split
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=44)
for train_idx, test_idx in gss.split(x, y, groups=groups):
    X_train, X_test = x.iloc[train_idx], x.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

# Train
randomForestRegModel = RandomForestRegressor(n_estimators=100, random_state=44)
randomForestRegModel.fit(X_train, y_train)

RandomForestRegressor(random_state=44)

**Printing both training sets**

In [ ]:
# Check performance
#print("Train R^2:", randomForestRegModel.score(X_train, y_train))
#print("Test R^2:", randomForestRegModel.score(X_test, y_test))
print(X_train)
print(y_train)

      dealer_id  is_stock_low  scheme_avalable
0          4886             1                1
1          4886             1                1
2          4886             1                1
3          4886             1                1
4          4886             1                1
...         ...           ...              ...
3695  232945855             0                1
3696  232947047             1                1
3697  232947047             1                1
3698  232947047             1                1
3699  232947047             1                1

[2928 rows x 3 columns]
0        4
1        4
2        4
3        4
4       30
        ..
3695    37
3696    23
3697    23
3698    23
3699    23
Name: DayDiff, Length: 2928, dtype: int64


**Checking importance of features**

In [ ]:
# Feature importance
importances = randomForestRegModel.feature_importances_
feature_importance = dict(zip(features, importances))
print("Feature importances:")
for feature, importance in feature_importance.items():
    print(f"{feature}: {importance:.4f}")

Feature importances:
dealer_id: 0.9022
is_stock_low: 0.0351
scheme_avalable: 0.0628


**Predicting data**

In [ ]:
import datetime
from datetime import timedelta


# Predict for dealer_id = 4886 (latest) with is_stock_low = 0
dealer_data = invoice_wise_stock_bal[invoice_wise_stock_bal['dealer_id'] == 4886].sort_values('purchase_date', ascending=False).iloc[0:1].copy()
last_purchase = datetime.datetime(2024, 11, 27)


def getPrediction(is_stock_low,scheme_available):
  dealer_data['is_stock_low'] = is_stock_low
  dealer_data['scheme_avalable'] = scheme_available

  X_new_0 = dealer_data[features]

  #print('X_new_0: ',X_new_0)
  last_purchase = dealer_data['purchase_date'].iloc[0]
  predicted_next_daydiff = float(randomForestRegModel.predict(X_new_0)[0])

  return predicted_next_daydiff

# Calculate predicted day differences
next_day_diff_00 = getPrediction(0, 0)  # High stock & no scheme
next_day_diff_10 = getPrediction(1, 0)  # Low stock & no scheme
next_day_diff_11 = getPrediction(1, 1)  # Low stock & scheme available
next_day_diff_01 = getPrediction(0, 1)  # High stock & scheme available

# Calculate predicted dates
predicted_date_00 = last_purchase + timedelta(days=next_day_diff_00)  # High stock & no scheme
predicted_date_10 = last_purchase + timedelta(days=next_day_diff_10)  # Low stock & no scheme
predicted_date_11 = last_purchase + timedelta(days=next_day_diff_11)  # Low stock & scheme available
predicted_date_01 = last_purchase + timedelta(days=next_day_diff_01)  # High stock & scheme available

# Output
print("High stock & no scheme:")
print("Predicted NextDayDiff:", round(next_day_diff_00, 2))
print("Last purchase date:", last_purchase)
print("Predicted next purchase date:", predicted_date_00)
print("--------------------------------------------------")

print("\nLow stock & no scheme:")
print("Predicted NextDayDiff:", round(next_day_diff_10, 2))
print("Last purchase date:", last_purchase)
print("Predicted next purchase date:", predicted_date_10)
print("--------------------------------------------------")

print("\nLow stock & scheme available:")
print("Predicted NextDayDiff:", round(next_day_diff_11, 2))
print("Last purchase date:", last_purchase)
print("Predicted next purchase date:", predicted_date_11)
print("--------------------------------------------------")

print("\nHigh stock & scheme available:")
print("Predicted NextDayDiff:", round(next_day_diff_01, 2))
print("Last purchase date:", last_purchase)
print("Predicted next purchase date:", predicted_date_01)

X_new_0:        dealer_id  is_stock_low  scheme_avalable
3071  232839435             0                0
X_new_0:        dealer_id  is_stock_low  scheme_avalable
3071  232839435             1                0
X_new_0:        dealer_id  is_stock_low  scheme_avalable
3071  232839435             1                1
X_new_0:        dealer_id  is_stock_low  scheme_avalable
3071  232839435             0                1
High stock & no scheme:
Predicted NextDayDiff: 99.35
Last purchase date: 2024-11-27 00:00:00
Predicted next purchase date: 2025-03-06 08:29:00.514286
--------------------------------------------------

Low stock & no scheme:
Predicted NextDayDiff: 47.42
Last purchase date: 2024-11-27 00:00:00
Predicted next purchase date: 2025-01-13 10:09:19.248958
--------------------------------------------------

Low stock & scheme available:
Predicted NextDayDiff: 90.63
Last purchase date: 2024-11-27 00:00:00
Predicted next purchase date: 2025-02-25 15:05:09.977062
-------------------------

In [ ]:
import datetime
from datetime import timedelta


# Predict for dealer_id = 4886 (latest) with is_stock_low = 0
dealer_data = invoice_wise_stock_bal.sort_values('purchase_date', ascending=False).iloc[0:1].copy()
last_purchase = datetime.datetime(2024, 11, 27)


def getPrediction(is_stock_low,scheme_available):
  dealer_data['is_stock_low'] = is_stock_low
  dealer_data['scheme_avalable'] = scheme_available

  X_new_0 = dealer_data[features]

  #print('X_new_0: ',X_new_0)
  last_purchase = dealer_data['purchase_date'].iloc[0]
  predicted_next_daydiff = float(randomForestRegModel.predict(X_new_0)[0])

  return predicted_next_daydiff

# Calculate predicted day differences
next_day_diff_00 = getPrediction(0, 0)  # High stock & no scheme
next_day_diff_10 = getPrediction(1, 0)  # Low stock & no scheme
next_day_diff_11 = getPrediction(1, 1)  # Low stock & scheme available
next_day_diff_01 = getPrediction(0, 1)  # High stock & scheme available

# Calculate predicted dates
predicted_date_00 = last_purchase + timedelta(days=next_day_diff_00)  # High stock & no scheme
predicted_date_10 = last_purchase + timedelta(days=next_day_diff_10)  # Low stock & no scheme
predicted_date_11 = last_purchase + timedelta(days=next_day_diff_11)  # Low stock & scheme available
predicted_date_01 = last_purchase + timedelta(days=next_day_diff_01)  # High stock & scheme available

# Output
print("High stock & no scheme:")
print("Predicted NextDayDiff:", round(next_day_diff_00, 2))
print("Last purchase date:", last_purchase)
print("Predicted next purchase date:", predicted_date_00)
print("--------------------------------------------------")

print("\nLow stock & no scheme:")
print("Predicted NextDayDiff:", round(next_day_diff_10, 2))
print("Last purchase date:", last_purchase)
print("Predicted next purchase date:", predicted_date_10)
print("--------------------------------------------------")

print("\nLow stock & scheme available:")
print("Predicted NextDayDiff:", round(next_day_diff_11, 2))
print("Last purchase date:", last_purchase)
print("Predicted next purchase date:", predicted_date_11)
print("--------------------------------------------------")

print("\nHigh stock & scheme available:")
print("Predicted NextDayDiff:", round(next_day_diff_01, 2))
print("Last purchase date:", last_purchase)
print("Predicted next purchase date:", predicted_date_01)


With is_stock_low = 0:
Predicted NextDayDiff: 95.66
Last purchase date: 2024-11-27 00:00:00
Predicted next purchase date: 2025-03-02 15:54:12.685714

With is_stock_low = 1:
Predicted NextDayDiff: 27.1
Last purchase date: 2024-11-27 00:00:00
Predicted next purchase date: 2024-12-24 02:22:01.714286
